In [1]:
import sys
import os
import gzip
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
def get_median(a):
    a.sort()
    centerIndex = len(a) // 2 
    if len(a)% 2 == 1:
        return a[centerIndex ]
    return ((a[centerIndex - 1] + a[centerIndex]) / 2)
from matplotlib_venn import venn3, venn3_circles
import matplotlib.pyplot as plt 
from matplotlib_venn import venn2, venn2_circles
from tqdm import tqdm

### Germline concordance pre-process

In [2]:
sample_name_list = [line.rstrip('\n') for line in open('/home/goldpm1/Meningioma/script/sample_name.txt', 'r')]
HC_PATH="/home/goldpm1/Meningioma/06.hc/03.HF"

input_file = []
for Sample_ID in sample_name_list:
    for TISSUE in ["Blood", "Tumor", "Dura"]: # 각각에 대해서 다 돌려보고 germline concordance도 확인해보자
        input_dir = HC_PATH + "/" + Sample_ID + "/" + TISSUE
        input_file.append (input_dir + "/" + Sample_ID + "_" + TISSUE + ".DP100.vcf")

input_file.sort()


whole_dic = {}
name_list = []
for file in tqdm(input_file):
    name = file.split("/")[-1].split(".")[0]
    name_list.append(name)
    whole_dic[name] = set()
    
    f = open( file, 'r' )
    for fline in f:
        if fline.startswith('#'):
            continue
        ftab = fline.rstrip().split('\t')
        ID = ftab[0] + '_' + ftab[1]    #"CHR.POS"
        whole_dic[name].add(ID)
    f.close()

print (name_list)

100%|██████████| 9/9 [00:01<00:00,  5.73it/s]

['220930_Blood', '220930_Dura', '220930_Tumor', '221026_Blood', '221026_Dura', '221026_Tumor', '221102_Blood', '221102_Dura', '221102_Tumor']


### Germline concordance running

In [3]:
first_list = list(name_list)
second_list = list(name_list)

output_dir = "/home/goldpm1/Meningioma/script/04.Shared_calls"
os.makedirs(output_dir, exist_ok = True)

fo = open(output_dir + "/germline_concordance.txt",'w')
fo.write('concor' + '\t')
for first in first_list:
    if first == first_list[-1]:
        fo.write(first + '\n')
    else:
        fo.write(first + '\t')

for first in first_list:
    fo.write(first + '\t')
    for second in second_list:
        if second == second_list[-1]:
            if first == second:
                fo.write('100' + '\n')
                print(first, second, '100')
            else:
                concordance = round(len(whole_dic[first] & whole_dic[second])/len(whole_dic[first] | whole_dic[second]) * 100, 2)
                fo.write(str(concordance) + '\n')
                print(first, second, str(concordance))

        else:
            if first == second:
                fo.write('100' + '\t')
                print(first, second, '100')
            else:
                concordance = round(len(whole_dic[first] & whole_dic[second])/len(whole_dic[first] | whole_dic[second]) * 100, 2)
                fo.write(str(concordance) + '\t')
fo.close()

220930_Blood 220930_Blood 100
220930_Blood 221102_Tumor 52.44
220930_Dura 220930_Dura 100
220930_Dura 221102_Tumor 51.43
220930_Tumor 220930_Tumor 100
220930_Tumor 221102_Tumor 52.15
221026_Blood 221026_Blood 100
221026_Blood 221102_Tumor 51.85
221026_Dura 221026_Dura 100
221026_Dura 221102_Tumor 51.94
221026_Tumor 221026_Tumor 100
221026_Tumor 221102_Tumor 51.59
221102_Blood 221102_Blood 100
221102_Blood 221102_Tumor 96.02
221102_Dura 221102_Dura 100
221102_Dura 221102_Tumor 95.78
221102_Tumor 221102_Tumor 100


In [4]:
cor_df = pd.read_csv (output_dir + "/germline_concordance.txt", sep ="\t")
cor_df.style.background_gradient(cmap ='coolwarm').set_precision(1)

# corr = round ( result.NUM_CLONE_record.iloc[ : , 1:].astype ( int).corr(),  2)
# corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

/home/goldpm1/miniconda3/envs/cnvpytor/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  


,concor,220930_Blood,220930_Dura,220930_Tumor,221026_Blood,221026_Dura,221026_Tumor,221102_Blood,221102_Dura,221102_Tumor
0,220930_Blood,100.0,95.5,96.5,52.3,52.4,52.1,51.8,51.6,52.4
1,220930_Dura,95.5,100.0,95.0,52.2,52.5,52.3,52.0,52.1,51.4
2,220930_Tumor,96.5,95.0,100.0,52.0,52.2,51.9,51.6,51.5,52.1
3,221026_Blood,52.3,52.2,52.0,100.0,96.2,96.0,52.0,52.0,51.9
4,221026_Dura,52.4,52.5,52.2,96.2,100.0,96.1,52.1,52.2,51.9
5,221026_Tumor,52.1,52.3,51.9,96.0,96.1,100.0,51.9,51.8,51.6
6,221102_Blood,51.8,52.0,51.6,52.0,52.1,51.9,100.0,97.2,96.0
7,221102_Dura,51.6,52.1,51.5,52.0,52.2,51.8,97.2,100.0,95.8
8,221102_Tumor,52.4,51.4,52.1,51.9,51.9,51.6,96.0,95.8,100.0
